# Overview

**ETL** : Extract, Transform, Load Project

**Extract**: extract data from different sources, read the data, and store into a dataframe.

**Transform**: clean, structure, and transform the raw data that has been extracted from the sources into a specific schema, so it meets the operation needs.

**Load**: load and store the data into a database for future analysis.

Data Sources:
1. Minimum Wage Data: https://www.kaggle.com/datasets/lislejoem/us-minimum-wage-by-state-from-1968-to-2017

1. Energy Data: https://www.kaggle.com/datasets/lislejoem/us_energy_census_gdp_10-14?resource=download

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, inspect

# Extract

## Store CSV into DataFrame

In [ ]:
#Load in energy data
csv_file_1 = "resources/energy_census_and_economic_data_US_2010-2014.csv"
energy_df = pd.read_csv(csv_file_1)
energy_df.head()

In [ ]:
#Load in minimum wage data
csv_file_2 = "resources/minimum_wage_data.csv"
minimum_wage_df = pd.read_csv(csv_file_2,encoding='ISO-8859-1')
minimum_wage_df.head()

# Transform

In [ ]:
#Cleaning Energy Consumption Data
year = []
state = []
energy_consumption = []
gdp = []

year_starter = 2010
year_counter = 0


while True:
    for index, row in energy_df.iterrows():
        if (index == 51):
            year.append(year_starter + year_counter)
            state.append(energy_df['State'][index])
            energy_consumption.append(energy_df[f"TotalC{year_starter + year_counter}"][index])
            gdp.append(energy_df[f"GDP{year_starter + year_counter}"][index])
            year_counter += 1
            continue

        else:
            year.append(year_starter + year_counter)
            state.append(energy_df['State'][index])
            energy_consumption.append(energy_df[f"TotalC{year_starter + year_counter}"][index])
            gdp.append(energy_df[f"GDP{year_starter + year_counter}"][index])
    
    if (year_starter + year_counter == 2015):
        break

In [ ]:
energy_df_cleaned = pd.DataFrame({
#     'year_state': year_state,
    'year': year,
    'state': state,
    'total_energy': energy_consumption ,
    'gdp': gdp
})

energy_df_cleaned

In [ ]:
#Cleaning Minimum Wage Data
# minimum_wage_df_cleaned = minimum_wage_df[['Year', 'State', 'State.Minimum.Wage', 'State.Minimum.Wage.2020.Dollars']].copy()
# minimum_wage_df_cleaned.head()
reduced_min_wage_data_df = minimum_wage_df[['Year', 'State', 'State.Minimum.Wage', 'CPI.Average']].copy()
renamed_min_wage_data_df = reduced_min_wage_data_df.rename(columns={'Year':'year', 'State':'state', 'State.Minimum.Wage': 'state_minimum_wage', 'CPI.Average':'cpi_average'})

cleaned_min_wage_data_df = renamed_min_wage_data_df.loc[  (renamed_min_wage_data_df['year'] > 2009) & (renamed_min_wage_data_df['year'] < 2015)]
cleaned_min_wage_data_df

## Create database connection

In [ ]:
protocol = 'postgresql'
username = 'postgres'
password = 'bootcamp'
host = 'localhost'
port = 5432
database_name = 'energy_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
# Confirm tables
inspector = inspect(engine)
inspector.get_table_names()

## Load DataFrames into database

In [ ]:
energy_df_cleaned.to_sql(name='energy_consumption', con=engine, if_exists='append', index=False)

In [ ]:
cleaned_min_wage_data_df.to_sql(name='minimum_wage', con=engine, if_exists='append', index=False)

## Confirm data has been added by querying the DB table

In [ ]:
pd.read_sql_query('select * from energy_consumption', con=engine).head()

In [ ]:
pd.read_sql_query('select * from minimum_wage', con=engine).head()